<a href="https://colab.research.google.com/github/alfazick/AppliedLLMCourse/blob/main/ProjectMultiModalBaseLineEval1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 21 November
# Plan
# 1) how IOU works and calculated and ok so today I want to understand

# 2) run IOU on default dataset to prepare evaluation pipiline (score should be best right?)

# 3) run inference on base model without fintuning and pass through evaluation pipeline

# 4) try to quantize the base model and check if it will perform similarly

# 5) prepare dataset for finetunung.




In [ ]:
# Clean slate and install stable versions
!pip uninstall transformers trl datasets bitsandbytes peft qwen-vl-utils wandb accelerate -y

!pip install transformers==4.46.2 \
             accelerate==1.0.1 \
             peft==0.13.2 \
             bitsandbytes>=0.45.0 \
             datasets==3.0.2 \
             trl==0.11.4 \
             qwen-vl-utils==0.0.8 \
             wandb==0.18.5

Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Successfully uninstalled transformers-4.46.2
Found existing installation: trl 0.11.4
Uninstalling trl-0.11.4:
  Successfully uninstalled trl-0.11.4
Found existing installation: datasets 3.0.2
Uninstalling datasets-3.0.2:
  Successfully uninstalled datasets-3.0.2
Found existing installation: bitsandbytes 0.48.2
Uninstalling bitsandbytes-0.48.2:
  Successfully uninstalled bitsandbytes-0.48.2
Found existing installation: peft 0.13.2
Uninstalling peft-0.13.2:
  Successfully uninstalled peft-0.13.2
Found existing installation: qwen-vl-utils 0.0.8
Uninstalling qwen-vl-utils-0.0.8:
  Successfully uninstalled qwen-vl-utils-0.0.8
Found existing installation: wandb 0.18.5
Uninstalling wandb-0.18.5:
  Successfully uninstalled wandb-0.18.5
Found existing installation: accelerate 1.0.1
Uninstalling accelerate-1.0.1:
  Successfully uninstalled accelerate-1.0.1


In [ ]:
# Cell 2 - MUST restart kernel
import os
os._exit(0)  # This forces kernel restart

In [ ]:
import os
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128,expandable_segments:True"
os.environ["PYTORCH_NO_CUDA_MEMORY_CACHING"] = "1"

In [ ]:
import torch
print("alloc conf:", os.environ.get("PYTORCH_CUDA_ALLOC_CONF"))


alloc conf: max_split_size_mb:128,expandable_segments:True


In [ ]:
import importlib

# Verify installed versions match requirements
requirements = {
    'torch': '2.4.1',
    'transformers': '4.46.2',
    'trl': '0.11.4',
    'datasets': '3.0.2',
    'bitsandbytes': '0.44.1',
    'peft': '0.13.2',
    'qwen_vl_utils': '0.0.8',
    'wandb': '0.18.5',
    'accelerate': '1.0.1'
}

print("=== VERSION CHECK ===")
for package, expected in requirements.items():
    try:
        module = importlib.import_module(package)
        actual = getattr(module, '__version__', 'unknown')
        status = "✅" if actual == expected else "⚠️"
        print(f"{status} {package}: {actual} (expected: {expected})")
    except ImportError:
        print(f"❌ {package}: NOT INSTALLED (expected: {expected})")

=== VERSION CHECK ===
⚠️ torch: 2.8.0+cu128 (expected: 2.4.1)
✅ transformers: 4.46.2 (expected: 4.46.2)
✅ trl: 0.11.4 (expected: 0.11.4)
✅ datasets: 3.0.2 (expected: 3.0.2)
⚠️ bitsandbytes: 0.48.2 (expected: 0.44.1)
✅ peft: 0.13.2 (expected: 0.13.2)
⚠️ qwen_vl_utils: unknown (expected: 0.0.8)
✅ wandb: 0.18.5 (expected: 0.18.5)
✅ accelerate: 1.0.1 (expected: 1.0.1)


In [ ]:
# 1) metric IoU (Interection over Union)
# measures how much two bounding boxes overlap
# It's a core metric for evaluation of object detection

# Math : Area of Overlap/Area of Union
# Intersection / (Box1 + Box2 - Intersection)

import torch
from torchvision import ops

# Boxes format: [x1, y1, x2, y2] where (x1,y1) is top-left, (x2,y2) is bottom-right
# make sure to get this format
predicted_boxes = torch.tensor([
    [2, 2, 10, 10],
])

ground_truth_boxes = torch.tensor([
    [3, 3, 11, 11],
])

iou_matrix = ops.box_iou(predicted_boxes,ground_truth_boxes)

In [ ]:
iou_matrix

tensor([[0.6203]])

In [ ]:

import inspect
source = inspect.getsource(ops.box_iou)
print(source)

def box_iou(boxes1: Tensor, boxes2: Tensor) -> Tensor:
    """
    Return intersection-over-union (Jaccard index) between two sets of boxes.

    Both sets of boxes are expected to be in ``(x1, y1, x2, y2)`` format with
    ``0 <= x1 < x2`` and ``0 <= y1 < y2``.

    Args:
        boxes1 (Tensor[N, 4]): first set of boxes
        boxes2 (Tensor[M, 4]): second set of boxes

    Returns:
        Tensor[N, M]: the NxM matrix containing the pairwise IoU values for every element in boxes1 and boxes2
    """
    if not torch.jit.is_scripting() and not torch.jit.is_tracing():
        _log_api_usage_once(box_iou)
    inter, union = _box_inter_union(boxes1, boxes2)
    iou = inter / union
    return iou



In [ ]:
from torchvision.ops.boxes import _box_inter_union
source = inspect.getsource(_box_inter_union)
print(source)

def _box_inter_union(boxes1: Tensor, boxes2: Tensor) -> tuple[Tensor, Tensor]:
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = _upcast(rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    return inter, union



In [ ]:
from torchvision.ops.boxes import _box_inter_union
source = inspect.getsource(_box_inter_union)
print(source)

def _box_inter_union(boxes1: Tensor, boxes2: Tensor) -> tuple[Tensor, Tensor]:
    area1 = box_area(boxes1)
    area2 = box_area(boxes2)

    lt = torch.max(boxes1[:, None, :2], boxes2[:, :2])  # [N,M,2]
    rb = torch.min(boxes1[:, None, 2:], boxes2[:, 2:])  # [N,M,2]

    wh = _upcast(rb - lt).clamp(min=0)  # [N,M,2]
    inter = wh[:, :, 0] * wh[:, :, 1]  # [N,M]

    union = area1[:, None] + area2 - inter

    return inter, union



In [ ]:
from torchvision.ops.boxes import box_area

source = inspect.getsource(box_area)
print(source)

def box_area(boxes: Tensor) -> Tensor:
    """
    Computes the area of a set of bounding boxes, which are specified by their
    (x1, y1, x2, y2) coordinates.

    Args:
        boxes (Tensor[N, 4]): boxes for which the area will be computed. They
            are expected to be in (x1, y1, x2, y2) format with
            ``0 <= x1 < x2`` and ``0 <= y1 < y2``.

    Returns:
        Tensor[N]: the area for each box
    """
    if not torch.jit.is_scripting() and not torch.jit.is_tracing():
        _log_api_usage_once(box_area)
    boxes = _upcast(boxes)
    return (boxes[:, 2] - boxes[:, 0]) * (boxes[:, 3] - boxes[:, 1])



In [ ]:
# ok let's verify manually
width = 10 - 2 # 8
height = 10 - 2 # 8

area_predict = width*height
width = 11 - 3
height = 11 - 3

area_truth = width*height
print(area_predict,area_truth)

64 64


In [ ]:
# ouch :)) cv vs cartesian
# ok so what confused me is naming conventions
# like lt = left top, and rb = right bottom
# but in high school math I get used to math coordinates
# like lb = left bottom and rt = righ top

# math should work probably anyway is just internal mental confusion


In [ ]:
# Standard Image

# (0,0) ────────► x increases
#   │
#   │    [x1, y1] = TOP-LEFT
#   │    [x2, y2] = BOTTOM-RIGHT
#   ▼
# y increases

# Cartesian
# y increases
#   ▲
#   │    [x1, y1] = BOTTOM-LEFT
#   │    [x2, y2] = TOP-RIGHT
#   │
# (0,0) ────────► x increases

In [ ]:
left_botom = max([2,2],[3,3])

In [ ]:
left_botom

[3, 3]

In [ ]:
right_top = min([10,10],[11,11])

In [ ]:
right_top

[10, 10]

In [ ]:
WidthHeight = [10 - 3, 10 - 3]

In [ ]:
WidthHeight

[7, 7]

In [ ]:
inter = WidthHeight[0]*WidthHeight[1]
union = area_predict + area_truth - inter

In [ ]:
union


79

In [ ]:
iou = inter / union # 49 / 79 # ok so 79 is a prime so no further simplification

In [ ]:
# ok so basically iou is a proportion of intersection area over (total_area - interection)

In [ ]:
iou

0.620253164556962

In [ ]:
print(ops.box_iou(predicted_boxes,ground_truth_boxes))

tensor([[0.6203]])


In [ ]:
# ok so rounded to up to 4 decimal, actually it's funny that code does it show it ?

iou = ops.box_iou(predicted_boxes,ground_truth_boxes)

iou.item()

0.6202531456947327

In [ ]:
# oh ok, so it's just print statemt which rounds it


In [ ]:
# ok it seems that I understand how individual IOU is calculated
# I guess for multiple images I should like average them
# or like average worst 100 examples ???
# or may be worst 25 % ?
# width area for experemients

# IOU Exploration Done
############################################


In [ ]:
# Task 2) run IOU on default dataset to prepare evaluation pipiline (score should be best right?)


In [ ]:
from datasets import load_dataset
dataset_id = "openfoodfacts/nutrition-table-detection"

nutrition_data = load_dataset(dataset_id)

In [ ]:
nutrition_data

DatasetDict({
    train: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'meta', 'objects'],
        num_rows: 1083
    })
    val: Dataset({
        features: ['image_id', 'image', 'width', 'height', 'meta', 'objects'],
        num_rows: 123
    })
})

In [ ]:
# Ok so what if there are multiple boxes ????
# how I am supposed to identify which box belongs to what
# ok this is going to be very fun
# what is going to be my strategy
# let me find how many data points have multiple boxes

nutrition_data["train"][0]



{'image_id': '0009800892204_1',
 'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>,
 'width': 2592,
 'height': 1944,
 'meta': {'barcode': '0009800892204',
  'off_image_id': '1',
  'image_url': 'https://static.openfoodfacts.org/images/products/000/980/089/2204/1.jpg'},
 'objects': {'bbox': [[0.057098764926195145,
    0.014274691231548786,
    0.603501558303833,
    0.991126537322998]],
  'category_id': [0],
  'category_name': ['nutrition-table']}}

In [ ]:

for point in nutrition_data["train"]:
    boxes = point["objects"]["bbox"]
    n = len(boxes) # multiple boxes in single image
    if n > 1:
        print(n)

2
3
2
2
2
2
2
3
2
2
2
2
2
2
2
2
2
2
2
3
2
3
3
2
2
2
2
3
2
2
2
2
3
3
2
2
2
2


In [ ]:
# ok it's not that bad as i expected but still
# like ok so I guess my initial strategy will be
# calculate like kind of matrix and then keep best IOU haha


# from scipy.optimize import linear_sum_assignment
# ok it's called hungarian algorithm :))
# not that dramatic for my 2-3 boxes
# but in case more than 20


# ok now it raises another question similat to my chess pawns
# like what if number of boxes don't match
# like there are 5 pawns and it only predicts 4
# what I am supposed to do :))
# I guess beside iou I will just report misses

In [ ]:
# ok so let's keep prepping the data
# {'image_id': '0009800892204_1',
#  'image': <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=2592x1944>,
#  'width': 2592,
#  'height': 1944,
#  'meta': {'barcode': '0009800892204',
#   'off_image_id': '1',
#   'image_url': 'https://static.openfoodfacts.org/images/products/000/980/089/2204/1.jpg'},
#  'objects': {'bbox': [[0.057098764926195145,
#     0.014274691231548786,
#     0.603501558303833,
#     0.991126537322998]],
#   'category_id': [0],
#   'category_name': ['nutrition-table']}}

# ok so it screams dictionary, like I guess I will iterate through whole dataset
# and create a ground truth dictionary with "image_id" as a key and list of boxes as values


# so I am aiming next evaluation loop

# for image_id,image_data in dataset:
#     predicted_boxes = model_predict(actual_image,prompt)
#     groundt_truth = ground_boxes[image_id]
#     iou_matrix = get_iou(predicted_boxes,ground_truth)

# haha I think I will write get_iou manually just to make sure I understood it correctly
# If I have a time :))


In [ ]:
# ok assume dataset will have image_id and objects
def populate_data(populating_dic,dataset,key="train"):
    if populating_dic is None:
        populating_dic = dict()

    if key not in dataset:
        print("dataset missing key",key)
        return None


    for point in dataset[key]:
        image_id = point["image_id"]
        raw_boxes = point["objects"]["bbox"]
        # eliminate the problem at the source


        # Fix format immediately: [y,x,y,x] → [x,y,x,y]
        # https://huggingface.co/datasets/openfoodfacts/nutrition-table-detection
        # bbox: List of bounding boxes in the format (y_min, x_min, y_max, x_max).
        # Coordinates are normalized between 0 and 1, using the top-left corner as the origin.

        raw_boxes = point["objects"]["bbox"]
        boxes = torch.as_tensor(
            [
                [x_min, y_min, x_max, y_max]
                for y_min, x_min, y_max, x_max in raw_boxes
            ],
            dtype=torch.float32,
            device="cpu",                # <-- IMPORTANT I caught a bug this data was on gpu
            # so when I decided to load Qwen, I got OOM :)))
        )

        category = point["objects"]["category_name"]
        # image_data = point["image"] # could not keep image too expensive

        populating_dic[image_id] = {

            "boxes":boxes,
            "category":category,
            ##"image_data":image_data

        }

    return populating_dic




In [ ]:
ground_truth = populate_data(None, nutrition_data, key="train")
# ok set and forget :)) ground truth should be solid


In [ ]:
def show_gpu(tag):
    print(f"[{tag}] allocated={torch.cuda.memory_allocated()/1024**3:.2f} GB, "
          f"reserved={torch.cuda.memory_reserved()/1024**3:.2f} GB")


In [ ]:
show_gpu("after populate")

[after populate] allocated=0.00 GB, reserved=0.00 GB


In [ ]:
cnt = 0
for p in ground_truth:
    if cnt == 5:
        break

    print(ground_truth[p])
    cnt +=1

{'boxes': tensor([[0.0143, 0.0571, 0.9911, 0.6035]]), 'category': ['nutrition-table']}
{'boxes': tensor([[0.1471, 0.1520, 0.5163, 0.5882]]), 'category': ['nutrition-table']}
{'boxes': tensor([[0.0101, 0.0083, 0.9783, 0.9944]]), 'category': ['nutrition-table']}
{'boxes': tensor([[0.3202, 0.1718, 0.6279, 0.6250]]), 'category': ['nutrition-table']}
{'boxes': tensor([[0.1432, 0.2512, 0.3766, 0.4683]]), 'category': ['nutrition-table']}


In [ ]:
# ok now I do dumb thing, but I want to make
# a sanity check, if there are multiple boxes
# per image so I will catch it here
# rather than try to debug it down the line done it before not doing it again :))
!pip install scipy

In [ ]:
# ok my mega check function
from scipy.optimize import linear_sum_assignment
from torchvision import ops
def evaluate(dict_predict,dict_ground_truth):
    all_ious = list()

    total_missed_real_boxes = 0
    total_extra_predicted = 0


    common_ids = set(dict_predict.keys()) & set(dict_ground_truth.keys())

    for image_id in common_ids:
        prediction_boxes = dict_predict[image_id]["boxes"]
        ground_truth_boxes = dict_ground_truth[image_id]["boxes"]

        total_predicted_boxes = len(prediction_boxes)
        total_ground_truth_boxes = len(ground_truth_boxes)

        # Calculate IoU matrix
        iou_matrix = ops.box_iou(prediction_boxes,ground_truth_boxes)
        # Hungarian matching
        pred_idx, gt_idx = linear_sum_assignment(-iou_matrix.numpy())

        # Get matched IoUs
        matched_ious = iou_matrix[pred_idx, gt_idx]
        all_ious.extend(matched_ious.tolist())

        # i have a bit doubt about this lines may be abs
        #
        total_missed_real_boxes += total_ground_truth_boxes - len(pred_idx)
        total_extra_predicted += total_predicted_boxes - len(matched_ious)

    return {
        'mean_iou': sum(all_ious) / len(all_ious),
        'missed_gt': total_missed_real_boxes,
        'extra_pred': total_extra_predicted
    }











In [ ]:
results = evaluate(ground_truth, ground_truth)
print(results)  # Should be: mean_iou=1.0, missed_gt=0, extra_pred=0

# cool solid results

{'mean_iou': 1.0, 'missed_gt': 0, 'extra_pred': 0}


In [ ]:
import gc
import time

def clear_memory():
    # Delete variables if they exist in the current global scope
    if 'inputs' in globals(): del globals()['inputs']
    if 'model' in globals(): del globals()['model']
    if 'processor' in globals(): del globals()['processor']
    if 'trainer' in globals(): del globals()['trainer']
    if 'peft_model' in globals(): del globals()['peft_model']
    if 'bnb_config' in globals(): del globals()['bnb_config']
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

clear_memory()



GPU allocated memory: 0.00 GB
GPU reserved memory: 0.00 GB


In [ ]:
!nvidia-smi

Fri Nov 21 23:53:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 5090        On  |   00000000:01:00.0 Off |                  N/A |
|  0%   32C    P1             65W /  575W |     506MiB /  32607MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:


# 3) run inference on base model without fintuning and pass through evaluation pipeline

In [ ]:
# 0) load model and tokenizer

# 1) I need to prepare messages per image ?
# 2) write a function that given single image returns boxes prediction
# a) I have a strong feeling that since response will be like string
# I will need some unified parsing function
# 3) Prepare prediction_dict, let's say 10 and run full evaluation


In [ ]:
%%capture
!pip install matplotlib
!pip install hf_transfer
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline


In [ ]:
import torch
import os
from matplotlib import pyplot as plt
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor,AutoProcessor
from qwen_vl_utils import vision_process,process_vision_info
import peft
import bitsandbytes as bnb

In [ ]:
# dummy run as before
messages = [
{
    "role":"user",
    "content":[
        {
            "type":"image",
            "image":"https://images.chesscomfiles.com/uploads/v1/images_users/tiny_mce/Ognian_Mikov/php2nnXz9.png",

        },
        {
            "type":"text","text":"Provide surrounding boxes for all pawns in this image and respond in <|bbox|> format"
        },
    ]
}
]

In [ ]:
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")


text = processor.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True
)

In [ ]:
text

'<|im_start|>system\nYou are a helpful assistant.<|im_end|>\n<|im_start|>user\n<|vision_start|><|image_pad|><|vision_end|>Provide surrounding boxes for all pawns in this image and respond in <|bbox|> format<|im_end|>\n<|im_start|>assistant\n'

In [ ]:
print("allocated:", torch.cuda.memory_allocated()/1024**3, "GB")
print("reserved:", torch.cuda.memory_reserved()/1024**3, "GB")


allocated: 15.830080032348633 GB
reserved: 30.099609375 GB


In [ ]:
image_inputs,video_inputs = process_vision_info(messages)
inputs = processor(text=[text],images=image_inputs,videos=video_inputs,padding=True,return_tensors="pt")
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct", torch_dtype=torch.bfloat16, device_map="cuda"
)
model.eval()
inputs = inputs.to("cuda")
generated_ids = model.generate(**inputs, max_new_tokens=128)
# Step 1: Trimming - Remove the input prompt from output
generated_ids_trimmed = [
    out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
]

output_text = processor.batch_decode(
    generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
)
print(output_text)


`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.73 GiB. GPU 0 has a total capacity of 31.37 GiB of which 674.69 MiB is free. Including non-PyTorch memory, this process has 30.70 GiB memory in use. Of the allocated memory 30.06 GiB is allocated by PyTorch, and 37.25 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)